In [1]:
from pathlib import Path
import os
import sys
import gc
import re
import shutil
import json
import math
import jinja2
from collections import defaultdict
import numpy as np
import pandas as pd
import bitsandbytes
import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from scml import pandasx as pdx
import lalaes2 as mylib
print(f"accelerate={accelerate.__version__}, bitsandbytes={bitsandbytes.__version__}")

accelerate=0.30.1, bitsandbytes=0.43.1


In [2]:
class ModelConf(NamedTuple):
    directory: Path
    model_max_length: int
    char_limit: int


input_filename = "persuade_02.parquet"
llm_key = "Qwen2-1.5B-Instruct"
llm_map: Dict[str, ModelConf] = {
    "gemma-1.1-2b-it": ModelConf(
        directory=Path("huggingface/google/gemma-1.1-2b-it"),
        model_max_length=8192,
        char_limit=4000,
    ),
    "Qwen2-1.5B-Instruct": ModelConf(
        directory=Path("huggingface/Qwen/Qwen2-1.5B-Instruct"),
        model_max_length=8192,
        char_limit=4000,
    ),
}
mc = llm_map[llm_key]
critique_col = f"ctq_{llm_key}"
environment = jinja2.Environment()
basic = mylib.BasicPreprocessor()

In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [4]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:1")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [5]:
df = pd.read_parquet(Path(f"input/{input_filename}"))
#df = df.sample(frac=0.001)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25996 entries, 0 to 25995
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   25996 non-null  object
 1   score      25996 non-null  int8  
 2   topic      25996 non-null  object
 3   full_text  25996 non-null  object
dtypes: int8(1), object(3)
memory usage: 634.8+ KB


# LLM-as-a-Judge

LLM takes on the role of a teacher to generate essay critique.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(mc.directory, model_max_length=mc.model_max_length)
tokenizer.pad_token = tokenizer.eos_token
print(f"""{tokenizer}
model_input_names={tokenizer.model_input_names}
pad_token_id={tokenizer.pad_token_id}
""")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2TokenizerFast(name_or_path='huggingface/Qwen/Qwen2-1.5B-Instruct', vocab_size=151643, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|im_end|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'attention_mask']
pad_token_id=151645



In [7]:
%%time
if "gemma" in str(mc.directory):
    model = AutoModelForCausalLM.from_pretrained(
        mc.directory,
        device_map="auto",
        torch_dtype=torch.float16,
        revision="float16",
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        mc.directory,
        torch_dtype="auto",
        device_map="auto"
    )

CPU times: user 4.65 s, sys: 2.98 s, total: 7.62 s
Wall time: 1min 4s


In [8]:
print(model)
print(model.config)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear

In [9]:
template = environment.from_string(
"""You are a teacher grading a student's essay. Assign a score on a scale of 1 to 6 based on the following rubric. A score of 6 indicates clear and consistent mastery with minor errors, showcasing insightful development of a viewpoint, outstanding critical thinking, strong use of appropriate examples and evidence, well-organized structure with coherence and smooth progression of ideas, skillful language use with varied vocabulary and sentence structure, and minimal errors in grammar, usage, and mechanics. Scores decrease with occasional errors (score of 5), lapses in quality (score of 4), weaknesses in critical thinking and organization (score of 3), serious flaws in viewpoint and coherence (score of 2), and fundamental errors hindering meaning (score of 1).
In your response, output only the score without explanation.
[ESSAY] {{ essay }} [/ESSAY]
"""
)

In [10]:
def critique(row) -> str:
    """Inference method and response extraction based on https://huggingface.co/Qwen/Qwen2-7B-Instruct"""
    prompt = template.render(essay=str(row["full_text"])[:mc.char_limit])
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer(text, truncation=True, return_tensors="pt").to(device)
    model_outputs = model.generate(
        **model_inputs, 
        max_new_tokens=16,
        do_sample=False,
        #temperature=0.01,
        #top_p=0.95,
        #top_k=40,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, model_outputs)]
    res = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    res = basic(res)
    if len(res)==1 and res.isdigit():
        res = f"SCORE OF {res}"
    return res


df[critique_col] = df.progress_apply(critique, axis=1)

  0%|                                                                                                                                                             | 0/25996 [00:00<?, ?it/s]/home/pankun/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pankun/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/pankun/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag 

In [11]:
pdx.value_counts(df[critique_col])

,count,percent
ctq_Qwen2-1.5B-Instruct,,
SCORE OF 6,16553,0.636752
SCORE OF 5,5905,0.227150
SCORE OF 1,1155,0.044430
SCORE OF 2,915,0.035198
SCORE OF 3,881,0.033890
SCORE OF 4,578,0.022234
C,5,0.000192
Score: 2,2,0.000077
SCORE OF 7,1,0.000038


# Review Data

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25996 entries, 0 to 25995
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   essay_id                 25996 non-null  object
 1   score                    25996 non-null  int8  
 2   topic                    25996 non-null  object
 3   full_text                25996 non-null  object
 4   ctq_Qwen2-1.5B-Instruct  25996 non-null  object
dtypes: int8(1), object(4)
memory usage: 837.9+ KB


In [13]:
cols = [critique_col, "score", "full_text"]
df[cols].head(10)

,ctq_Qwen2-1.5B-Instruct,score,full_text
0,SCORE OF 5,3,"Phones Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation. Driving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems. That's why there's a thing that's called no texting while driving. That's a really important thing to remember. Some people still do it because they think It's stupid. No matter what they do they still have to obey it because that's the only way how did he save. Sometimes on the news there is either an accident or a suicide. It might involve someone not looking where they're going or tweet that someone sent. It either injury or death. If a mysterious number says I'm going to kill you and they know where you live but you don't know the person's contact ,It makes you puzzled and make you start to freak out. Which can end up really badly. Phones are fine to use and it's also the best way to come over help. If you go through a problem and you can't find help you ,always have a phone there with you. Even though phones are used almost every day as long as you're safe it would come into use if you get into trouble. Make sure you do not be like this phone while you're in the middle of driving. The news always updated when people do something stupid around that involves their phones. The safest way is the best way to stay safe."
1,SCORE OF 6,4,"This essay will explain if drivers should or should not be able to use electronic devices while operating a vehicle. Using a phone while driving can cause very bad wrecks, because they are putting themselves and other people at risk. People should not be able to use cell phones while operating a vehicle is because bad wrecks, putting others' lives in danger, and may cause death. First of all, wrecks can be caused by looking down at your phone while driving. Most importantly, you should always keep your eyes directly on the road when behind the wheel of a car. On that note, you will have an outrageously car payment just because you were looking down at your phone. Moreover, if you the reason the accident happened the person that was operating the other car can sue you for lots of money, and you will have to pay it. Therefore, if you don't pay whatever the person charges you consequences will be served and that's behind bars. Another reason why we should not be able to use cell phones while operating an automobile is that you are putting other people's lives in danger. Thus, looking down at your cell phone you can get someone else hurt for the uncommunicative act you committed. As well as unconsciousness, injuries, or being in a hospital. The main key to not texting and driving while behind the wheel of a car. Mainly, it can cause tragic, terrifying, and horrific things to you and to others and that's death. That's the most important thing to not using electronic devices while operating a vehicle. With the intension, of you not keeping your eyes and staying focus on the road you can easily turn off the road or hit another victim. Possibly another humankind could go away of all flesh because of your stupidity. Must be remembered, always stay focus on the road so you can get to your destination safely. So, you can't cause accidents, put other people lives in danger, and possib

In [14]:
df.to_parquet(f"output/{input_filename}", index=False)
assert df.notna().all(axis=None)

In [15]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 4:47:48.842835
